In [2]:
from sklearn.neighbors import KNeighborsClassifier
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../DataProcessing/')
import SplitTVT as tvt


In [3]:
trainLoader, valLoader, testLoader = tvt.load_data("../Data/Dataset/", 512)

ValueError: expected sequence of length 1250 at dim 2 (got 70)

In [ ]:
len(trainLoader.dataset)